In [1]:
from HestonFFT import Call_Heston
import numpy as np 
import scipy 
import math
import scipy.integrate
import pandas as pd
import torch
import torch.nn as nn
from NN import ModelLarge
from TestPerformance import test_performanceNN
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
#loading the training set
df = pd.read_csv("HestonTrainXXL.csv").to_numpy()[:,1:]
y = df[:,0].reshape(-1,1)
x = df[:,1:]
train_x = torch.from_numpy(x).double()
train_y = torch.from_numpy(y).reshape(-1,1).double() 

In [3]:
#setting the hyperparameters of the model 
input_dim = 8
hidden_dim = 128

model = ModelLarge(input_dim, hidden_dim).to(device)
model = model.double()

#loss and learning rate 
criterion = nn.L1Loss()
learning_rate = 0.001

#optimizer
optimizer = torch.optim.Adamax(model.parameters(), lr = learning_rate) 
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
                            optimizer, factor=0.5, 
                            patience=3, 
                            verbose=True ) 


#valutation set 
df = pd.read_csv("HestonTestNN.csv").to_numpy()[:,1:]
test_x = df[:,1:]
test_y = df[:,0]
test_y_Tensor = torch.from_numpy(test_y).to(device)
test_x_Tensor = torch.from_numpy(test_x).to(device)

In [11]:
train = False
if train: 
    %%time 
    num_epochs = 300
    iter = 0
    N = train_x.shape[0]
    batch = 100

    train_x = train_x.to(device)
    train_y = train_y.to(device)

    for epoch in range(num_epochs):
        index = torch.randperm(N)
        train_input_shuffled = train_x[index]
        train_target_shuffled = train_y[index]
        s = 0

        for b in range(0, N, batch):
            x = train_x.narrow(0, b, batch)
            y = train_y.narrow(0, b, batch)

            # Clear gradients w.r.t. parameters
            optimizer.zero_grad()

            # Forward pass to get output/logits
            outputs = model(x)
            # print(y, outputs)
            # Calculate Loss:
            loss = criterion(outputs, y)
            # print(loss)
            s += loss

            # Getting gradients w.r.t. parameters
            loss.backward()

            # Updating parameters
            optimizer.step()

            iter += 1
        if epoch % 2 == 0:
            test = f"Test Loss:  {test_performanceNN(test_x_Tensor , test_y_Tensor , model).item():.2e}"
            print(f"Epoch: {epoch}    Loss: {s.mean()}    {test}")
        # scheduler wll reduce the learning rate if gor 5 epoch there is no gain in term of the loss
        scheduler.step(s.mean())


Epoch: 0    Loss: 12.627512205976977    Test Loss:  3.49e-03
Epoch: 2    Loss: 3.676419880430981    Test Loss:  2.04e-03
Epoch: 4    Loss: 3.1567868716615934    Test Loss:  2.20e-03
Epoch: 6    Loss: 3.0400011673440424    Test Loss:  3.15e-03
Epoch: 8    Loss: 2.616211071862402    Test Loss:  1.02e-03
Epoch: 10    Loss: 2.537531762350157    Test Loss:  1.10e-03
Epoch: 12    Loss: 2.122584266000305    Test Loss:  1.97e-03
Epoch: 14    Loss: 2.080474675670011    Test Loss:  2.22e-03
Epoch: 16    Loss: 1.9172571622331374    Test Loss:  1.26e-03
Epoch: 18    Loss: 1.9120964486428884    Test Loss:  8.51e-04
Epoch: 20    Loss: 2.008193127598748    Test Loss:  2.81e-03
Epoch: 22    Loss: 1.6818483607980839    Test Loss:  1.68e-03
Epoch: 24    Loss: 1.6782398814774422    Test Loss:  1.55e-03
Epoch: 26    Loss: 1.9137522195086631    Test Loss:  1.76e-03
Epoch: 28    Loss: 1.6591076086264376    Test Loss:  7.12e-04
Epoch: 30    Loss: 1.641499524566649    Test Loss:  8.31e-04
Epoch: 32    Loss: 1

Epoch: 250    Loss: 0.16074647169147238    Test Loss:  1.80e-04
Epoch: 252    Loss: 0.15975813894869087    Test Loss:  1.80e-04
Epoch: 254    Loss: 0.16000039760240375    Test Loss:  1.79e-04
Epoch: 256    Loss: 0.15976167322124785    Test Loss:  1.85e-04
Epoch 00257: reducing learning rate of group 0 to 7.8125e-06.
Epoch: 258    Loss: 0.15566603537516768    Test Loss:  1.82e-04
Epoch: 260    Loss: 0.15536932603135234    Test Loss:  1.82e-04
Epoch: 262    Loss: 0.155248863175337    Test Loss:  1.82e-04
Epoch: 264    Loss: 0.1549065387238027    Test Loss:  1.83e-04
Epoch: 266    Loss: 0.15489060544208152    Test Loss:  1.82e-04
Epoch: 268    Loss: 0.15459567923530165    Test Loss:  1.83e-04
Epoch: 270    Loss: 0.15450894531803236    Test Loss:  1.82e-04
Epoch: 272    Loss: 0.15445950368832273    Test Loss:  1.83e-04
Epoch: 274    Loss: 0.15430608755632888    Test Loss:  1.81e-04
Epoch: 276    Loss: 0.1542745667143972    Test Loss:  1.80e-04
Epoch: 278    Loss: 0.15421343762899814    Tes

In [5]:
#saving the model
#torch.save(model, "modelNNLarge.pt")

In [4]:
#loading the model trained on 1milion observations  and look at the performance +
mod = torch.load("modelNNLarge.pt")

datasets = ["HestonTestS.csv", "HestonTestM.csv", "HestonTestL.csv"]
for dataset in datasets:
    print(dataset)
    df = pd.read_csv(dataset).to_numpy()[:,1:]
    X = df[:,1:]
    y = df[:,0]
    test_y_Tensor = torch.from_numpy(y).to(device)
    test_x_Tensor = torch.from_numpy(X).to(device)
    test_performanceNN(test_x_Tensor, test_y_Tensor, mod, type_ = "both", to_return = False, time_ = True)
    print("\n")


HestonTestS.csv
Time:  0.012998819351196289
AAE:  6.09e-05
MAE:  1.46e-03


HestonTestM.csv
Time:  0.02700042724609375
AAE:  5.69e-05
MAE:  8.60e-04


HestonTestL.csv
Time:  0.05307912826538086
AAE:  5.28e-05
MAE:  9.55e-04




In [5]:
#loading the model trained on 100k observations  and look at the performance +
mod = torch.load("modelNNLarge_SmallTrain.pt")

datasets = ["HestonTestS.csv", "HestonTestM.csv", "HestonTestL.csv"]
for dataset in datasets:
    print(dataset)
    df = pd.read_csv(dataset).to_numpy()[:,1:]
    X = df[:,1:]
    y = df[:,0]
    test_y_Tensor = torch.from_numpy(y).to(device)
    test_x_Tensor = torch.from_numpy(X).to(device)
    test_performanceNN(test_x_Tensor, test_y_Tensor, mod, type_ = "both", to_return = False, time_ = True)
    print("\n")


HestonTestS.csv
Time:  0.019085168838500977
AAE:  1.78e-04
MAE:  2.34e-03


HestonTestM.csv
Time:  0.026999711990356445
AAE:  1.76e-04
MAE:  1.34e-03


HestonTestL.csv
Time:  0.054001808166503906
AAE:  1.71e-04
MAE:  2.67e-03


